# alpha

In [1]:
import numpy as np
import pandas as pd
from zipline.research import returns

In [2]:
stocks = [str(i) for i in range(300001,300021)]

In [3]:
mu = returns(stocks, '2018-3-20','2018-5-31').mean()

[2018-06-04 14:50:27.262232] INFO: zipline.finance.metrics.tracker: 模拟1个交易日
首个开盘时间: 2018-05-31 01:31:00+00:00
最后收盘时间: 2018-05-31 07:00:00+00:00


In [4]:
mu

特锐德(300001)    -0.001444
神州泰岳(300002)   -0.002906
乐普医疗(300003)    0.004536
南风股份(300004)   -0.013959
探路者(300005)    -0.000831
莱美药业(300006)   -0.001242
汉威科技(300007)   -0.001003
天海防务(300008)   -0.004395
安科生物(300009)    0.000766
立思辰(300010)     0.006260
鼎汉技术(300011)   -0.005505
华测检测(300012)    0.003423
新宁物流(300013)   -0.002562
亿纬锂能(300014)   -0.000749
爱尔眼科(300015)    0.004830
北陆药业(300016)    0.004430
网宿科技(300017)   -0.003557
中元股份(300018)   -0.002182
硅宝科技(300019)   -0.004257
银江股份(300020)   -0.001900
dtype: float64

# 初始权重

In [5]:
init_assets = mu.iloc[2:8]

In [6]:
w_data = np.random.randn(len(init_assets))

In [7]:
init_w = pd.Series(w_data.T / sum(w_data), index = init_assets.index)

In [8]:
init_w

乐普医疗(300003)    0.754246
南风股份(300004)    0.842851
探路者(300005)    -1.780067
莱美药业(300006)   -0.665538
汉威科技(300007)    1.233536
天海防务(300008)    0.614973
dtype: float64

# 优化

In [9]:
from zipline.optimize.objectives import TargetWeights, MaximizeAlpha
from zipline.optimize.constraints import (MaxGrossExposure, NetExposure,
                                          NotExceed, NotLessThan,
                                          DollarNeutral, FixedWeight)
from zipline.optimize.core import run_optimization, calculate_new_weights

## 最大alpha

In [10]:
obj = MaximizeAlpha(mu)

In [11]:
me = MaxGrossExposure(1.5)
ne = NotExceed(0.4)
fw = FixedWeight(mu.index[2],0.3)

In [12]:
calculate_new_weights(obj, [me,ne,fw], init_w)

特锐德(300001)    -0.0
神州泰岳(300002)   -0.0
乐普医疗(300003)    0.3
南风股份(300004)   -0.4
探路者(300005)    -0.0
莱美药业(300006)   -0.0
汉威科技(300007)   -0.0
天海防务(300008)   -0.0
安科生物(300009)    0.0
立思辰(300010)     0.4
鼎汉技术(300011)   -0.4
华测检测(300012)    0.0
新宁物流(300013)   -0.0
亿纬锂能(300014)   -0.0
爱尔眼科(300015)    0.0
北陆药业(300016)    0.0
网宿科技(300017)   -0.0
中元股份(300018)   -0.0
硅宝科技(300019)   -0.0
银江股份(300020)   -0.0
dtype: float64

## 目标权重

In [13]:
target_weights = pd.Series([0.1,-0.3,-0.2,0.7],index=mu.index[4:8])

In [14]:
target_weights

探路者(300005)     0.1
莱美药业(300006)   -0.3
汉威科技(300007)   -0.2
天海防务(300008)    0.7
dtype: float64

In [15]:
obj = TargetWeights(target_weights)

In [16]:
calculate_new_weights(obj, [me,ne], init_w)

乐普医疗(300003)    0.0
南风股份(300004)    0.0
探路者(300005)     0.1
莱美药业(300006)   -0.3
汉威科技(300007)   -0.2
天海防务(300008)    0.4
dtype: float64

In [17]:
from toolz import concat
import cvxpy as cvx

## 可解情形

In [18]:
objective = MaximizeAlpha(mu)
me = MaxGrossExposure(1.5)
ne = NotExceed(0.4)
fw = FixedWeight(mu.index[2],0.3)
constraints = [me,ne]

In [19]:
cvx_objective = objective.to_cvxpy(init_w)
new_weights = objective.new_weights
new_weights_series = objective.new_weights_series
constraint_map = {
    c: c.to_cvxpy(new_weights, new_weights_series, init_w)
    for c in constraints
}
cvx_constraints = list(concat(constraint_map.values()))

In [23]:
problem = cvx.Problem(cvx_objective, cvx_constraints)
problem.solve()

0.01173867217044785

In [24]:
new_weights.value

array([-7.85244334e-10, -1.93639781e-09,  1.32003964e-09, -3.99999999e-01,
       -4.32292255e-10, -6.63800408e-10, -5.27224512e-10, -2.96415058e-09,
        3.97299800e-10,  3.99999992e-01, -3.99999986e-01,  2.49847970e-09,
       -1.60897951e-09, -3.88228080e-10,  2.99999962e-01,  2.84048279e-09,
       -2.64927256e-09, -1.29377376e-09, -3.10403912e-09, -1.08524316e-09])

In [25]:
problem.solver_stats.solver_name

'ECOS'

# 无解问题

## 冲突限制

In [26]:
objective = MaximizeAlpha(mu)
me = MaxGrossExposure(1.5)
dn = DollarNeutral(0.001)
nl = NotLessThan(0.5)  # 单个权重不少于0.5与每个权重不得超过0.4矛盾
ne = NotExceed(0.4)
fw = FixedWeight(mu.index[2],0.3)
constraints = [me, dn, ne, nl, fw]

In [27]:
cvx_objective = objective.to_cvxpy(init_w)
new_weights = objective.new_weights
new_weights_series = objective.new_weights_series
constraint_map = {
    c: c.to_cvxpy(new_weights, new_weights_series, init_w)
    for c in constraints
}
cvx_constraints = list(concat(constraint_map.values()))

In [28]:
infeasible_problem = cvx.Problem(cvx_objective, cvx_constraints)
infeasible_problem.solve(verbose=True)


ECOS 2.0.4 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -9.682e-04  -7.519e+00  +2e+02  7e-01  6e-01  1e+00  2e+00    ---    ---    1  1  - |  -  - 
 1  +4.133e-03  -1.651e+00  +4e+01  7e-01  2e-01  2e+00  4e-01  0.7997  9e-02   1  0  0 |  0  0
 2  +1.276e-03  +8.561e+01  +3e+00  6e+00  6e-01  1e+02  3e-02  0.9890  5e-02   1  0  0 |  0  0
 3  +1.355e-03  +8.102e+03  +3e-02  6e+00  6e-01  8e+03  3e-04  0.9890  1e-04   1  0  0 |  0  0
 4  +1.390e-03  +7.206e+05  +4e-04  6e+00  6e-01  7e+05  3e-06  0.9890  1e-04   2  0  0 |  0  0
 5  +1.424e-03  +6.404e+07  +4e-06  6e+00  6e-01  6e+07  4e-08  0.9890  1e-04   1  0  0 |  0  0
 6  +1.463e-03  +5.705e+09  +4e-08  6e+00  6e-01  6e+09  4e-10  0.9890  1e-04   0  0  0 |  0  0

PRIMAL INFEASIBLE (within feastol=1.2e-10).
Runtime: 0.000536 seconds.



-inf

In [43]:
infeasible_problem.status

'infeasible'

## 问题数据分析

In [29]:
prob_data = infeasible_problem.get_problem_data(cvx.ECOS)

In [45]:
for d in prob_data:
    print(type(d))

<class 'dict'>
<class 'cvxpy.reductions.solvers.solving_chain.SolvingChain'>
<class 'list'>


### 数据1

In [46]:
data_1 = prob_data[0]

#### c

In [61]:
c = data_1['c']
# 长度=41
c

array([ 0.00144354,  0.00290633, -0.00453552,  0.01395945,  0.00083065,
        0.00124155,  0.001003  ,  0.00439518, -0.0007659 , -0.00625959,
        0.0055054 , -0.00342344,  0.0025617 ,  0.00074902, -0.00482965,
       -0.00443037,  0.00355693,  0.0021823 ,  0.00425695,  0.00189981,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ])

#### offset

In [62]:
data_1['offset']

array([0.])

#### dims

In [64]:
dims = data_1['dims']

In [66]:
dims.zero

1

#### A

In [68]:
A = data_1['A']

In [69]:
type(A)

scipy.sparse.csc.csc_matrix

In [70]:
A.data

array([1.])

#### b

In [71]:
b = data_1['b']

In [72]:
b

array([0.3])

#### G

In [74]:
G = data_1['G']

In [82]:
G.data

array([ 1., -1.,  1., -1., -1.,  1., -1.,  1., -1., -1.,  1., -1.,  1.,
       -1.,  1., -1., -1.,  1., -1.,  1., -1.,  1., -1., -1.,  1., -1.,
        1., -1.,  1., -1., -1.,  1., -1.,  1., -1.,  1., -1., -1.,  1.,
       -1.,  1., -1.,  1., -1., -1.,  1., -1.,  1., -1.,  1., -1., -1.,
        1., -1.,  1., -1., -1.,  1., -1.,  1., -1., -1.,  1., -1.,  1.,
       -1., -1.,  1., -1.,  1., -1., -1.,  1., -1.,  1., -1., -1.,  1.,
       -1.,  1., -1., -1.,  1., -1.,  1., -1., -1.,  1., -1.,  1., -1.,
       -1.,  1., -1.,  1., -1., -1.,  1., -1.,  1., -1., -1.,  1., -1.,
        1., -1., -1.,  1., -1.,  1., -1., -1., -1., -1.,  1., -1., -1.,
        1., -1., -1.,  1., -1., -1.,  1., -1., -1.,  1., -1., -1.,  1.,
       -1., -1.,  1., -1., -1.,  1., -1., -1.,  1., -1., -1.,  1., -1.,
       -1.,  1., -1., -1.,  1., -1., -1.,  1., -1., -1.,  1., -1., -1.,
        1., -1., -1.,  1., -1., -1.,  1., -1., -1.,  1., -1., -1.,  1.,
       -1., -1.,  1., -1., -1.,  1.])

In [79]:
G.get_shape()

(104, 41)

In [49]:
for k,v in data_1.items():
    print(k,v)

c [ 0.00144354  0.00290633 -0.00453552  0.01395945  0.00083065  0.00124155
  0.001003    0.00439518 -0.0007659  -0.00625959  0.0055054  -0.00342344
  0.0025617   0.00074902 -0.00482965 -0.00443037  0.00355693  0.0021823
  0.00425695  0.00189981  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.        ]
offset [0.]
dims <cvxpy.reductions.solvers.conic_solvers.conic_solver.ConeDims object at 0x7fc02504ed30>
A   (0, 2)	1.0
b [0.3]
G   (0, 0)	1.0
  (20, 0)	-1.0
  (44, 0)	1.0
  (45, 0)	-1.0
  (84, 0)	-1.0
  (1, 1)	1.0
  (21, 1)	-1.0
  (46, 1)	1.0
  (47, 1)	-1.0
  (85, 1)	-1.0
  (2, 2)	1.0
  (22, 2)	-1.0
  (41, 2)	1.0
  (42, 2)	-1.0
  (48, 2)	1.0
  (49, 2)	-1.0
  (86, 2)	-1.0
  (3, 3)	1.0
  (23, 3)	-1.0
  (41, 3)	1.0
  (42, 3)	-1.0
  (50, 3)	1.0
  (51, 3)	-1.0
  (87, 3)	-1.0
  (4, 4)	1.0
  :	:
  (40, 32)	1.0
  (13, 33)	-1

In [52]:
dims = data_1['dims']

In [54]:
dims.nonpos

104

In [56]:
len(data_1['h'])

104

In [34]:
import re

In [35]:
PATTERN = re.compile('new_weights\[\d{1,}\]')

In [36]:
re.compile(r'(?<=送)(\d+(?:\.\d+)?)\D')

re.compile(r'(?<=送)(\d+(?:\.\d+)?)\D', re.UNICODE)

In [37]:
re.match(PATTERN, 'new_weights[0] + -0.4 <= 0')

<_sre.SRE_Match object; span=(0, 14), match='new_weights[0]'>

In [38]:
for c in problem.constraints:
    print(c)

Sum(abs(new_weights), None, False) + -1.5 <= 0
abs(Sum(reshape(  (0, 2)	1.0
  (1, 3)	1.0
  (2, 4)	1.0
  (3, 5)	1.0
  (4, 6)	1.0
  (5, 7)	1.0 * reshape(new_weights, (20,)), (6,)), None, False)) + -0.001 <= 0
new_weights[0] + -0.4 <= 0
-0.4 + -new_weights[0] <= 0
new_weights[1] + -0.4 <= 0
-0.4 + -new_weights[1] <= 0
new_weights[2] + -0.4 <= 0
-0.4 + -new_weights[2] <= 0
new_weights[3] + -0.4 <= 0
-0.4 + -new_weights[3] <= 0
new_weights[4] + -0.4 <= 0
-0.4 + -new_weights[4] <= 0
new_weights[5] + -0.4 <= 0
-0.4 + -new_weights[5] <= 0
new_weights[6] + -0.4 <= 0
-0.4 + -new_weights[6] <= 0
new_weights[7] + -0.4 <= 0
-0.4 + -new_weights[7] <= 0
new_weights[8] + -0.4 <= 0
-0.4 + -new_weights[8] <= 0
new_weights[9] + -0.4 <= 0
-0.4 + -new_weights[9] <= 0
new_weights[10] + -0.4 <= 0
-0.4 + -new_weights[10] <= 0
new_weights[11] + -0.4 <= 0
-0.4 + -new_weights[11] <= 0
new_weights[12] + -0.4 <= 0
-0.4 + -new_weights[12] <= 0
new_weights[13] + -0.4 <= 0
-0.4 + -new_weights[13] <= 0
new_weights[14]

In [39]:
for c in problem.constants():
    print(c.value)

[-0.00144354 -0.00290633  0.00453552 -0.01395945 -0.00083065 -0.00124155
 -0.001003   -0.00439518  0.0007659   0.00625959 -0.0055054   0.00342344
 -0.0025617  -0.00074902  0.00482965  0.00443037 -0.00355693 -0.0021823
 -0.00425695 -0.00189981]
1.5
  (0, 2)	1.0
  (1, 3)	1.0
  (2, 4)	1.0
  (3, 5)	1.0
  (4, 6)	1.0
  (5, 7)	1.0
0.001
0.4
-0.4
0.4
-0.4
0.4
-0.4
0.4
-0.4
0.4
-0.4
0.4
-0.4
0.4
-0.4
0.4
-0.4
0.4
-0.4
0.4
-0.4
0.4
-0.4
0.4
-0.4
0.4
-0.4
0.4
-0.4
0.4
-0.4
0.4
-0.4
0.4
-0.4
0.4
-0.4
0.4
-0.4
0.4
-0.4
0.5
0.3


In [40]:
c.expr.name()

'0.3'

In [41]:
cv = c.expr.constants()[0]

In [42]:
cv.value

array(0.3)